In [1]:
import sys
import json
from flask import Flask,render_template, request, redirect, url_for
from flask import jsonify
# for creating the mapper code
from sqlalchemy import Column, ForeignKey, Integer, String, ARRAY
from sqlalchemy.types import UserDefinedType
# for configuration and class code
from sqlalchemy.ext.declarative import declarative_base

# for creating foreign key relationship between the tables
from sqlalchemy.orm import relationship

# for configuration
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# create declarative_base instance
Base = Base2 = declarative_base()


#Class Definition
class Description(Base):
    __tablename__ = 'description'
    synset_id     = Column(Integer, primary_key=True)
    resource      = Column(String, nullable=False)
    label         = Column(String, nullable=False)
    description   = Column(String, nullable=False)
        
    @property
    def serialize(self):
        return {
        'resource'   : self.resource,
        'label'      : self.label,
        'description': self.description,
        'synset_id'  : self.synset_id,
     }
    
      

#Dictionary ={}
class Annotation_candidates(Base):
    __tablename__ ='annotation_candidate'
    id     = Column(Integer, primary_key=True)
    text   = Column(String, nullable=False)
    offset = Column(Integer, nullable=False)
    ressources_candidates = Column(String, nullable=False)
    
    @property
    def serialize(self):
        return {
        'text'                 : self.text,
        'offset'               : self.offset,
        'resources_candidates' : self.resources_candidates,
        'id'                   : self.id,
     }
    
# creates a create_engine instance at the bottom of the file
engine1 = create_engine('sqlite:///description.db?check_same_thread=False')
Base.metadata.create_all(engine1)

engine = create_engine('sqlite:///annotation_candidate.db?check_same_thread=False')
Base.metadata.create_all(engine)

In [2]:


#Methode definition
def get_synsets(w):
    syns = wordnet.synsets(w)
    return syns

def get_definitions(syns):
    return syns.definition()

def get_label(syns):
    return syns.lemmas()[0].name()

def get_source(syns):
    return syns.lexname()

def build_description(synset_id,resource,label,description):
    engine1 = create_engine('sqlite:///description.db?check_same_thread=False')
    Base.metadata.bind = engine1

    DBSession = sessionmaker(bind=engine1)
    session   =  DBSession()
   # orm.init_db('sqlite:////tmp/db.sqlite')
    
    global Description
    descr = Description(synset_id = synset_id, resource = resource, label = label, description = description)
    session.add(descr)
    #session.flush()
    session.commit()
    Description = descr.serialize()
    return jsonify(Description)

def build_annot_candidate(offset,text,ressources_candidates):
    engine = create_engine('sqlite:///annotation_candidate.db?check_same_thread=False')
    Base.metadata.bind = engine

    DBSession = sessionmaker(bind=engine)
    session =  DBSession()
    global AnnotationCandidate
    annotation_candidate = AnnotationCandidate(text = text, offset = offset, ressources_candidates = ressources_candidates)
    session.add(annotation_candidate)
    session.commit()
    return jsonify(AnnotationCandidate = annotation_candidate.serialize())


In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
#nltk.download('wordnet')
from nltk.corpus import wordnet #wornet loaded
import re

#nltk.download('stopwords')
#nltk.download('punkt')


def stop_words_filtering(file):
    reg_exp = r"[a-zA-Z0-9]+"
    #stop_words = []
    stop_words = set(stopwords.words('english')) #stopwords just for englisch text
    fobj = open(file, "r")
    word_tokens= []
    b = []
    a = re.compile(reg_exp)
    for line in fobj:
#example = "Pet food for shoes and fabric"    
        
        b = b + a.findall(line)
    
       # word_tokens = word_tokens + word_tokenize(line) 
        
        filtered_sentence =  [w for w in b if not w in stop_words] 
  
        filtered_sentence = [] 
  
        for w in b: #just filtern stopword in a text
            if w not in stop_words: 
                filtered_sentence.append(w) 
    fobj.close()            
    return filtered_sentence 
    

In [4]:
#b = stop_words_filtering("test.txt")

def get_description(w):
    
    synsets = get_synsets(w)
    i = 0
    for syns in synsets:
        build_description(i,get_source(syns),get_label(syns),get_definitions(syns))
        i = i+1
    description = session.query(Description).all()
    return description

def get_offset(w,file):
    with open(file,'r') as f:
        content = f.read()
        offset = content.index(w)
    f.close()
    return offset
def make_annotation(file):
    filtered_sentence = stop_words_filtering(file)
    for word in filtered_sentence:
        build_annot_candidate(get_offset, word, get_description(word))
    return 1


In [5]:
#make_annotation(test.txt)

In [6]:
app = Flask(__name__)



#api funktionen
def get_annotationCandidate():
    annotation_candidate = session.query(Annotation_candidate).all()
    return jsonify(annotation_candidate=[a.serialize for a in annotation_candidate])


def get_annotationCandidate(annot_id):
    annotation_candidate = session.query(Annotation_candidate).filter_by(id=annot_id).one()
    return jsonify(annotation_candidate=annotation_candidate.serialize)

@app.route('/')
def index():
  return 'Server Works done!'
  


@app.route('/')
@app.route('/annotApi/', methods=['GET'])
def annotsFunction():
    if request.method == 'GET':
        request.args.get(file = '' )
        return get_annotationCandidate(file)
    
@app.route('/annotApi')
def say_hello():
  annotation_candidate = session.query(Annotation_candidate).all()
  return render_template('annotation_candidate.html', annotation_candidate=annotation_candidate)
 # return 'Hello from Server'
    
#@app.route('/annotApi/<int:id>', methods=['GET'])
#nltkdef annotsFunctionId():
   # if request.method == 'GET':
       # return get_annotationCandidate(annot_id)
   
 #make_annotation(test.txt)
if __name__ == '__main__':
    make_annotation('test.txt')    
   # app.debug = True
    from werkzeug.serving import run_simple
    run_simple('localhost', 5000, app)


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: description.synset_id
[SQL: INSERT INTO description (synset_id, resource, label, description) VALUES (?, ?, ?, ?)]
[parameters: (0, 'noun.animal', 'pet', 'a domesticated animal kept for companionship or amusement')]
(Background on this error at: http://sqlalche.me/e/gkpj)